In [7]:
from resnet import ResNet18 
import torch 
import torch.nn as nn 

in_channels=3
activation=nn.LeakyReLU 
cnn_dim=8192 
out_features=1 
last_avg_kernel_size = 3
resnet = ResNet18(in_channels, activation, cnn_dim, out_features, last_avg_kernel_size)
resnet

ResNet18(
  (features): Sequential(
    (stage1): Sequential(
      (unit1): ResUnit(
        (body): ResBottleneck(
          (conv1): ConvBlock(
            (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): LeakyReLU(negative_slope=0.01)
          )
          (conv2): ConvBlock(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): LeakyReLU(negative_slope=0.01)
          )
          (conv3): ConvBlock(
            (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (identity_conv): ConvBlock(
          (conv): Conv2d(3, 256, kernel_size=(1, 1

In [2]:
x = torch.rand(1, in_channels, 32,32)
resnet(x)

tensor([[-0.3306]], grad_fn=<AddmmBackward0>)

In [3]:
import torchvision
valid_dataset  = torchvision.datasets.CIFAR10(root = '/data3/bumjin/bumjin_data/',  
                                                    train=False,  
                                                    download=True,
                                                    transform =torchvision.transforms.ToTensor()) 

Files already downloaded and verified


In [4]:
valid_dataset[0][0].size()

torch.Size([3, 32, 32])

In [8]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)
activation = Swish
out_features = 10
resnet = ResNet18(in_channels, activation, cnn_dim, out_features, last_avg_kernel_size)
x = torch.rand(1, in_channels, 32,32)
resnet(x)

tensor([[-0.0489, -0.2882,  0.5466, -0.0649,  0.3975,  0.4133, -0.4318,  0.2100,
         -0.2456,  0.3243]], grad_fn=<AddmmBackward0>)

In [10]:
resnet

ResNet18(
  (features): Sequential(
    (stage1): Sequential(
      (unit1): ResUnit(
        (body): ResBottleneck(
          (conv1): ConvBlock(
            (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): Swish()
          )
          (conv2): ConvBlock(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activ): Swish()
          )
          (conv3): ConvBlock(
            (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (identity_conv): ConvBlock(
          (conv): Conv2d(3, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): 

In [11]:
class Model(nn.Module):
    def __init__(self, in_channels, cnn_dim=256):
        super().__init__()
        self.cnn1 = nn.Sequential(nn.Conv2d(in_channels,16,5,2,4),   Swish())
        self.cnn2 = nn.Sequential(nn.Conv2d(16,32,3,2,1),  Swish())
        self.cnn3 = nn.Sequential(nn.Conv2d(32,64,3,2,1),  Swish())
        self.cnn4 = nn.Sequential(nn.Conv2d(64,64,3,2,1),  Swish())
        self.flatten = nn.Flatten()
        self.out = nn.Linear(cnn_dim, 1)
    
    def forward(self, x):
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x = self.cnn4(x)
        x = self.flatten(x)
        x = self.out(x)
        return x
    

In [12]:
model = Model()
model(x)

RuntimeError: Given groups=1, weight of size [16, 1, 5, 5], expected input[1, 3, 32, 32] to have 1 channels, but got 3 channels instead